In [1]:
from ttrtypes import Type, BType, PType, Pred, MeetType, JoinType, ListType, \
SingletonType, HypObj, LazyObj, FunType, RecType, Fun, Ty, Re, RecTy, Possibility, \
AbsPath, TTRString, TTRStringType, KPlusStringType
from records import Rec
from utils import show, example, ttrace, nottrace, ttracing, show_latex

# Basic Types

In [2]:
Ind = BType('Ind')

Ind.judge('j')
Ind.judge('k')
Ind.judge('l')
Ind.judge('m')

print(Ind.judge('j'))
print(Ind.judge('n'))
print(Ind.witness_cache)

True
True
['j', 'k', 'l', 'm', 'n']


In [3]:
print(Ind.query('h'))

False


In [4]:
print(Ind.create())

_a0


# PTypes

In [5]:
run = Pred('run',[Ind])
p = PType(run,['j'])

print(show(p))
print(p.validate())
show_latex(p)

run(j)
True


<IPython.core.display.Latex object>

In [6]:
p.create()

print(p.witness_cache)
print(Ind.judge_nonspec())
print(p.query_nonspec())
print(p.judge_nonspec())

['_e0']
True
True
True


In [7]:
man = Pred('man',[Ind])
manj = PType(man,['j'])
manj.create()

print(manj.witness_cache)

['_e1']


# Meet types

In [8]:
T1 = Type()
T2 = Type()
T1.judge('a')
T2.judge('a')
T1.judge('b')
T = MeetType(T1,T2)

print(T.query('a'))
print(T.query('b'))
print(T.judge_nonspec())

True
False
True


In [9]:
T11 = Type()
T21 = Type()
T31 = MeetType(T11,T21)

print(T31.judge_nonspec())
print(T31.witness_cache)
print(T11.witness_cache)
print(T21.witness_cache)
print(T31.create())

True
[]
['_a1']
['_a1']
_a2


# Join types

In [10]:
T3 = JoinType(T1,T2)

print(T3.witness_cache)
print(T3.query('a'))
print(T3.query('b'))
print(T3.create())
print(T3.witness_cache)
print(T1.witness_cache)
print(T2.witness_cache)

[]
True
True
_a3
['a', 'b', '_a3']
['a', 'b']
['a']


# The subtype relation

In [11]:
x = HypObj([Ind])

print(show(x.types))
print(x.validate())

[Ind]
True


In [12]:
T4 = Type()
x1 = T4.create_hypobj()

print(show(x1.types))

[T4]


In [13]:
x2 = T.create_hypobj()

print(show(x2.types))
print(T1.query(x2))

[T0, T1, (T0&T1)]
True


In [14]:
print(T1.subtype_of(T1))
print(T.subtype_of(T1))
print(T1.subtype_of(T3))
print(T3.subtype_of(T1))
print(T.subtype_of(T3))

True
True
True
False
True


# Learning witness conditions

In [15]:
ttrace()
T.learn_witness_condition(lambda x: x)
nottrace()

Meet types are logical and cannot learn new conditions


[]

# Functions and function types

In [16]:
f = Fun('x',Ind,'x')
print(f.app('j'))
show_latex(f)

j


<IPython.core.display.Latex object>

In [17]:
IndToInd = FunType(Ind,Ind)

print(IndToInd.query(f))
print(IndToInd.query_nonspec()) # Perhaps function types should be non-empty if their domain and range types are non-empty or if it's not the case that: the domain type is empty and the range type is non-empty (material conditional)

True
True


In [18]:
a = IndToInd.create_hypobj()
print(show(a))

h14


In [19]:
print(f.validate_arg('j'))
print(f.validate_arg('x'))

True
False


In [20]:
f1 = Fun('x',Ind,PType(run,['x']))
print(f1.show())

lambda x:Ind . run(x)


In [21]:
p1 = f1.app('j')
print(show(p1))

run(j)


In [22]:
p1c = f1.appc('j')
print(show(p1c))

run(j)


In [23]:
ttrace()
print(show(f1.appc('x')))
nottrace()

lambda x:Ind . run(x)(x): badly typed function application
None


[]

# The type $\textit{Type}$ (called `Ty` in `pyttr`)

In [24]:
print(Ty.query(T))
print(Ty.query(Ty))
print(Ty.query(p1))
print(Ty.query_nonspec()) # Presumably this should be True since at least Ty.query(Ty) returns True

True
True
True
True


In [25]:
ttrace()
Ty.learn_witness_condition(lambda x: x)
nottrace()

Ty is a logical type and cannot learn new conditions


[]

In [26]:
IndToTy = FunType(Ind,Ty)

print(IndToTy.query(f1))
print(IndToTy.query(f))

True
False


# Dependent types -- functions which return types

In [27]:
f2 = Fun('x',Ind,PType(man,['x']))
print(show(f2))

lambda x:Ind . man(x)


In [28]:
love = Pred('love',[Ind,Ind])
f3 = Fun('x',Ind,Fun('y',Ind,PType(love,['x','y'])))
p2 = f3.app('j').app('m')
print(show(p2))

love(j, m)


In [29]:
p3 = Fun('x',Ind,Fun('y',Ind,PType(love,['x','y'])).app('j')).app('m')
print(show(p3))

love(m, j)


# List types

In [30]:
ListInd = ListType(Ind)

print(ListInd.query(['j','k']))
print(show(ListInd))
print(ListInd.query([]))

True
[Ind]
True


# Singleton types

In [31]:
Ind_j = SingletonType(Ind,'j')

print(Ind_j.query('j'))
print(Ind_j.query('k'))
print(Ind_j.query('x'))
print(show(Ind_j))

True
False
False
Ind_j


In [32]:
Ind_jk = SingletonType(ListType(Ind),['j']+['k'])

print(show(Ind_jk))
print(Ind_jk.query(['j','k']))

[Ind]_[j, k]
True


In [33]:
Ind_appk = SingletonType(Ind,Fun('x',Ind,'j').app('k'))

print(show(Ind_appk))
print(Ind_appk.query('j'))

Ind_j
True


# Possibilities

In [34]:
print(Fun('x',Ind,Ind).app('j').query('k'))

True


In [35]:
print(Fun('x',Ind,PType(man,['x'])).app('j').query_nonspec())

True


In [36]:
print(show(Fun('x',Ind,PType(man,['x'])).app('j')))

man(j)


In [37]:
print(show(manj))

man(j)


In [38]:
print(manj.query_nonspec())

True


In [39]:
M = Possibility('M')
manj.in_poss(M)

print(Fun('x',Ind,PType(man,['x'])).app('j').in_poss(M).query_nonspec())

True


In [40]:
print(show(M))


M:
_____________________________________________
man(j): [_e1]
_____________________________________________



# Record types

In [41]:
print(Ty.query(RecType({'x':Ind})))

True


In [42]:
print(RecType({'x':Ind}).query(Rec({'x':'j'})))
print(RecType({'x':Ind}).query(Rec({'x':'j', 'y':'k','z':'e1'})))
print(RecType({'x':Ind, 'y':Ind}).query(Rec({'x':'j'}))) #False

True
True
False


In [43]:
print(RecTy.query(RecType({'x':Ind,
                           'c':(Fun('v',Ind,PType(man,['v'])), ['x'])})))

True


In [44]:
T_man = RecType({'x':Ind, 'c':(Fun('v',Ind,PType(man,['v'])), ['x'])})

print(T_man.query(Rec({'x':'j', 'c':'_e1'})))

True


In [45]:
Ind.in_poss(M)
T_man.in_poss(M)

print(show(M))


M:
_____________________________________________
man(j): [_e1]
Ind: [j, k, l, m, n, _a0]
_____________________________________________



In [46]:
print(T_man.in_poss(M).query(Rec({'x':'j', 'c':'_e1'})))

True


##### Using singleton types in manifest fields to ensure identity

In [47]:
print(RecType({'x':Ind, \
               'y':(Fun('v',Ind,SingletonType(Ind,'v')),['x'])})
      .in_poss(M)
      .query(Rec({'x':'j',
                  'y':'j',
                  'c':'e1'})))

True


In [48]:
print(show(T_man))
print(show(T_man.create())) 
print(show(M))

{x : Ind, c : (lambda v:Ind . man(v), [x])}
{x = _a4, c = _e2}

M:
_____________________________________________
man(j): [_e1]
Ind: [j, k, l, m, n, _a0, _a4]
man(_a4): [_e2]
_____________________________________________



In [49]:
print(T_man.query(T_man.create()))

True


In [50]:
print(RecTy.query(RecType({'x':Ind,
                           'c': RecType({'c': (Fun('v',Ind,PType(man,['v'])), ['x'])})}))) 

True


True, even though the path in the dependent field is not defined (too deeply embedded)

#### Subtyping for record types

In [51]:
print(T_man.subtype_of(T_man))

True


In [52]:
print(RecType({'x':Ind,
               'c': (Fun('v',Ind,PType(man,['v'])), ['x'])}).subtype_of(Re))

True


In [53]:
print(RecType({'x':Ind,
               'c': RecType({'c': (Fun('v',Ind,PType(man,['v'])), ['x'])})})
      .subtype_of(Re))  

True


#### Paths in dependent fields

In [54]:
print(show(RecType({'x':Ind,
                    'c': RecType({'c': (Fun('v',Ind,PType(man,['v'])), ['x'])})})))

print(RecType({'x':Ind,
               'c': RecType({'c': (Fun('v',Ind,PType(man,['v'])), ['x'])})})
      .validate()) 
 

{x : Ind, c : {c : (lambda v:Ind . man(v), [x])}}
False


False.  This would have been True in the Oz implementation since
 paths always started from the top of the record type.  Now paths
 start in the record type in which they occur.

In [55]:
print(T_man.validate())

print(RecType({'x':Ind,
               'e':(Fun('v',Ind,RecType({'e':PType(man,['v'])})), ['x'])})
      .validate())
# True. Here the function is at the right level. 

print(RecType({'x':Ind,
               'e':(Fun('v',Ind,RecType({'e':PType(man,['v'])})), ['x'])})
      .in_poss(M)
      .query(Rec({'x':'j',
                  'y':'j',
                  'e':Rec({'e':'_e1'})}))) 


True
True
True


In [56]:
print(RecType({'x':Ind,
               'y':(Fun('v',Ind,SingletonType(Ind,'v')),['x'])})
      .in_poss(M)
      .query(Rec({'x':'j',
                  'y':'j',
                  'e':Rec({'e':'_e1'})})))

print(RecType({'x':Ind,
               'y':(Fun('v',Ind,SingletonType(Ind,'v')),['x']),
               'e':(Fun('v',Ind,RecType({'e':PType(man,['v'])})), ['y'])})
      .in_poss(M).query(Rec({'x':'j',
                             'y':'j',
                             'e':Rec({'e':'_e1'})})))

print(show(RecType({'x':Ind,
                    'y':(Fun('v',Ind,SingletonType(Ind,'v')),['x']),
                    'e':(Fun('v',Ind,RecType({'e':PType(man,['v'])})), ['y'])})
           .in_poss(M)
           .create()))


True
True
{x = _a6, y = _a6, e = {e = _e4}}


In [57]:
print(RecType({'x':Ind,
               'y':(Fun('v',Ind,SingletonType(Ind,'v')),['x']),
               'e':(Fun('v',Ind,RecType({'e':PType(man,['v'])})), ['y'])})
      .in_poss(M)
      .judge(Rec({'x':'j',
                  'y':'j',
                  'e':Rec({'e':'_e1'})})))

True


In [58]:
T_a_man =  RecType({'x':Ind,
                    'y':(Fun('v',Ind,SingletonType(Ind,'v')),['x']),
                    'e':(Fun('v',Ind,RecType({'e':PType(man,['v'])})), ['y'])})

T_a_man.in_poss(M).judge(Rec({'x':'j',
                              'y':'j',
                              'e':Rec({'e':'_e1'})}))

print(show(T_a_man.witness_cache))


[{x = j, y = j, e = {e = _e1}}]


#### Tricky cases of subtyping

In [59]:
print(RecType({'x':Ind,
               'y':Ind})
      .subtype_of(RecType({'x':Ind})))

print(RecType({'x':Ind,
               'y':Ind,
               'e': (Fun('v',Ind,PType(man,['v'])),['y'])})
      .subtype_of(RecType({'x':Ind,
                           'y':(Fun('v',Ind,SingletonType(Ind,'v')),['x']),
                           'e': (Fun('v',Ind,PType(man,['v'])),['y'])}))) #False

print(RecType({'x':Ind,
               'y':(Fun('v',Ind,SingletonType(Ind,'v')),['x']),
               'e': (Fun('v',Ind,PType(man,['v'])),['y'])})
      .subtype_of(RecType({'x':Ind,
                           'y':Ind,
                           'e': (Fun('v',Ind,PType(man,['v'])),['y'])})))

True
False
True


In [60]:
R1 = RecType({'x':Ind,
              'y':(Fun('v',Ind,SingletonType(Ind,'v')),['x']),
              'e': (Fun('v',Ind,PType(man,['v'])),['y'])})

R2 = RecType({'x':Ind,
              'y':Ind,
              'e': (Fun('v',Ind,PType(man,['v'])),['y'])})

print(R1.subtype_of(R2))

print(RecType({'x' : R1})
      .subtype_of(RecType({'x' : R2})))

True
True


#### Validation of record types

In [61]:
print(RecTy.query(RecType({'x' : Ind,
                           'e' : (Fun('v',Ind,PType(man,['v'])),['x'])})))

print(RecType({'x' : Ind,
               'e' : (Fun('v',Ind,PType(man,['v'])),['x'])}).validate())

print(RecTy.query(RecType({'x' : Ty,
                           'e' : (Fun('v',Ind,PType(man,['v'])),['x'])})))
#In the earlier implementation this was false.  If we want to replicate this we can first validate as below.

print(RecType({'x' : Ty,
               'e' : (Fun('v',Ind,PType(man,['v'])),['x'])}).validate())

True
True
True
False


In [62]:
ttrace()
h1 = RecType({'x' : Ty,
              'e' : (Fun('v',Ind,PType(man,['v'])),['x'])}).create_hypobj()
print(show(h1))
nottrace()

lambda v:Ind . man(v)(h38): badly typed function application
Unresolved dependency in {x : Ty, e : (lambda v:Ind . man(v), [x])}
None


[]

In [63]:
print(RecType({'x' : RecType({'x' : Ind,
                              'e' : (Fun('v',Ind,PType(man,['v'])),['x'])})}).
            validate())

True


In [64]:
print(RecType({'x' : (Fun('v',Ind,RecType({'x' : Ind,
                                           'e' : PType(man,['v'])})),['x'])}).
            validate())

False


In [65]:
print(show(RecType({'x' : RecType({'x' : Ind,
                                   'e' : (Fun('v',Ind,PType(man,['v'])),['x'])})})
           .create()))

{x = {x = _a7, e = _e5}}


In [66]:
print(show(RecTy.create()))

_T0


In [67]:
print(RecType({'x' : RecType({'x' : Ind,\
                              'e' : (Fun('v',Ind,PType(man,['v'])),['y'])}),
                'y' : Ind}).
            validate()) #False

False


In [68]:
print(RecType({'x' : (Fun('v',Ind,RecType({'x' : Ind,
                                           'e' : PType(man,['v'])})),['y']),
               'y' : Ind}).
            validate())

True


In [69]:
print(show(RecType({'x' : (Fun('v',Ind,RecType({'x' : Ind,
                                           'e' : PType(man,['v'])})),['y']),
               'y' : Ind}).
            create()))


{y = _a8, x = {x = _a9, e = _e6}}


In [70]:
print(show(RecType({'x' : (Fun('v',Ind,RecType({'x' : Ind,
                                                'e' : PType(man,['v'])})),['y']),
                    'y' : Ind}).
            create_hypobj()))


{y = h46, x = {x = h47, e = h48}}


In [71]:
print(RecType({'x' : RecType({'x' : Ind,
                              'e' : (Fun('v',Ind,PType(man,['v'])),['x'])}),
               'y' : (Fun('v',Ind,SingletonType(Ind,'v')),['x.x'])}).
            validate()) 

True


In [72]:
print(RecType({'x' : (Fun('v',Ind,RecType({'x' : Ind,
                                           'e' : PType(man,['v'])})),['y']),
               'y' : (Fun('v',Ind,SingletonType(Ind,'v')),['x.x'])}).
            validate())

False


 The corresponding type in the previous implementation would have
 been ok since it would have been written (mutatis mutandis)
 
 `RecType({'x' : RecType({'x' : Ind,\
                         'e' : (Fun('v',Ind, PType(man,['v'])),['y'])}),\
          'y' : (Fun('v',Ind,SingletonType(Ind,'v')),['x.x'])})`
 
 and path names in the dependent fields would always have been
 interpreted from the top of the record.  So this type from the old
 system is not expressible if path names are interpreted locally.
 However, there is a type very like it in the two following cells:


In [73]:
print(RecType({'x' : (Fun('v',Ind,RecType({'x' : SingletonType(Ind,'v'),
                                           'e' : (Fun('v',Ind,PType(man,['v'])),['x'])})), ['y']),
               'y' : Ind}).
            validate())

True


In [74]:
print(show(RecType({'x' : (Fun('v',Ind,RecType({'x' : SingletonType(Ind,'v'),
                                                'e' : (Fun('v',Ind,PType(man,['v'])),['x'])})), ['y']),
               'y' : Ind}).
            create()))

print(show(RecType({'x' : (Fun('v',Ind,RecType({'x' : SingletonType(Ind,'v'),
                                                'e' : (Fun('v',Ind,PType(man,['v'])),['x'])})), ['y']),
               'y' : Ind}).
            create_hypobj()))

{y = _a10, x = {x = _a10, e = _e7}}
{y = h55, x = {x = h55, e = h56}}


In [75]:
print(RecType({'x' : (Fun('v',Ind,RecType({'x' : SingletonType(Ind,'v'),\
                                     'e' : (Fun('v',Ind,PType(man,['v'])),['x'])})), ['y']),
               'y' : Ind}).subtype_of(
RecType({'x' : RecType({'x' : Ind,
                        'e' : (Fun('v',Ind,PType(man,['v'])),['x'])}),
               'y' : Ind})
))

True


The reverse of the above is false:

In [76]:
print(
RecType({'x' : RecType({'x' : Ind,
                        'e' : (Fun('v',Ind,PType(man,['v'])),['x'])}),
         'y' : Ind}).subtype_of(
RecType({'x' : (Fun('v',Ind,RecType({'x' : SingletonType(Ind,'v'),
                                     'e' : (Fun('v',Ind,PType(man,['v'])),['x'])})), ['y']),
         'y' : Ind})
))

False


In [77]:
print(
RecType({'x' : (Fun('v',Ind,SingletonType(Ind,'v')),['y']),\
         'y' : (Fun('v',Ind,SingletonType(Ind,'v')),['x'])}).\
validate()
) #False

False


In [78]:
ttrace()
print(
RecType({'x' : (Fun('v',Ind,SingletonType(Ind,'v')),['y']),
         'y' : (Fun('v',Ind,SingletonType(Ind,'v')),['x'])}).
create()
) #None
nottrace()

y not a label in {}
lambda v:Ind . Ind_v(None): badly typed function application
x not a label in {}
lambda v:Ind . Ind_v(None): badly typed function application
Unresolved dependency in {x : (lambda v:Ind . Ind_v, [y]), y : (lambda v:Ind . Ind_v, [x])}
None


[]

In [79]:
ttrace()
print(
RecType({'x' : (Fun('v',Ind,SingletonType(Ind,'v')),['y']),
         'y' : (Fun('v',Ind,SingletonType(Ind,'v')),['x'])}).
create_hypobj()\
) #None
nottrace()


y not a label in {}
lambda v:Ind . Ind_v(None): badly typed function application
x not a label in {}
lambda v:Ind . Ind_v(None): badly typed function application
Unresolved dependency in {x : (lambda v:Ind . Ind_v, [y]), y : (lambda v:Ind . Ind_v, [x])}
None


[]

In [80]:
print(\
RecType({'x' : (Fun('v',Ind,SingletonType(Ind,'v')),['y']),
         'y' : (Fun('v',Ind,SingletonType(Ind,'v')),['x'])}).
query(\
Rec({'x' : 'j',\
     'y' : 'j'})\
)\
) # True!

True


In [81]:
print(\
RecType({'x' : (Fun('v',Ind,SingletonType(Ind,'v')),['y']),
         'y' : (Fun('v',Ind,SingletonType(Ind,'v')),['x'])}).
query(
Rec({'x' : 'j',
     'y' : 'k'})
)
) # False

False


 The five previous examples show that circularity in dependencies can be used
 in a limited fashion if desired.  Given a record that you are
 checking it is possible for the dependencies to be resolved using
 the record.  However, the type will not be validated and it is not
 possible to create objects of the type, which in turn means that it
 will not be possible to show that it is a subtype of another type.

#### Lazy objects in record types

In [82]:
print(
RecType({'x' : ListType(Ind),
         'y' : ListType(Ind),
         'z' : (Fun('v1',ListType(Ind),
                Fun('v2',ListType(Ind),SingletonType(ListType(Ind),LazyObj(['v1','+','v2'])))),['x','y'])}).
validate()
) 

True


In [83]:
print(
RecType({'x' : Ind,
         'y' : Ind,
         'e' : (Fun('v1',Ind,
                 Fun('v2',Ind,PType(love,['v1','v2']))), ['x','y'])}).
validate()
)

True


In [84]:
PType(love,['j','m']).in_poss(M).judge('e8')
print(
RecType({'x' : Ind,
         'y' : Ind,
         'e' : (Fun('v1',Ind,
                 Fun('v2',Ind,PType(love,['v1','v2']))), ['x','y'])}).
in_poss(M).
query(Rec({'x' : 'j',
       'y' : 'm',
       'e' : 'e8'}))
)

True


In [85]:
print(
RecType({'x' : ListType(Ind),
         'y' : ListType(Ind),
         'z' : (Fun('v1',ListType(Ind),
                Fun('v2',ListType(Ind),SingletonType(ListType(Ind),LazyObj(['v1','+','v2'])))),['x','y'])}).
query(
Rec({'x' : ['j'],
     'y' : ['m'],
     'z' : ['j', 'm']})
)
) 

True


In [86]:
print(
show(
RecType({'x' : ListType(Ind),
         'y' : ListType(Ind),
         'z' : (Fun('v1',ListType(Ind),
                Fun('v2',ListType(Ind),SingletonType(ListType(Ind),LazyObj(['v1','+','v2'])))),['x','y'])}).
create_hypobj()
)
)

{x = h67, y = h68, z = [h67, +, h68]}


In [87]:
print(
show(
RecType({'x' : SingletonType(ListType(Ind),['j']),
         'y' : ListType(Ind),
         'z' : (Fun('v1',ListType(Ind),
                Fun('v2',ListType(Ind),SingletonType(ListType(Ind),LazyObj(['v1','+','v2'])))),['x','y'])}).
create_hypobj()
)
)

{x = [j], y = h69, z = [[j], +, h69]}


In [88]:
print(
show(
RecType({'x' : SingletonType(ListType(Ind),['j']),
         'y' : SingletonType(ListType(Ind),['m']),
         'z' : (Fun('v1',ListType(Ind),
                Fun('v2',ListType(Ind),SingletonType(ListType(Ind),LazyObj(['v1','+','v2'])))),['x','y'])}).
create_hypobj()
)
)

{x = [j], y = [m], z = [j, m]}


In [89]:
print(
show(
RecType({'x' : FunType(Ind,RecTy),
         'y' : Ind,
         'z' : (Fun('f',FunType(Ind,RecTy), Fun('v',Ind,SingletonType(RecTy,LazyObj(['f','@','v'])))),
                 ['x','y'])}).
query(
Rec({'x' : Fun('v',Ind,RecType({'e' : PType(man, ['v'])})),
     'y' : 'j',
     'z' : RecType({'e' : PType(man, ['j'])})})
)
)
)

True


In [90]:
print(
show(
RecType({'x' : FunType(Ind,RecTy),
         'y' : Ind,
         'w' : Ind,
         'z' : (Fun('f',FunType(Ind,RecTy), Fun('v',Ind,SingletonType(RecTy,LazyObj(['f','@','v'])))),
                 ['x','y'])}).
subtype_of(
RecType({'x' : FunType(Ind,RecTy),
         'y' : Ind,
         'w' : Ind,
         'z' : (Fun('f',FunType(Ind,RecTy), Fun('v',Ind,SingletonType(RecTy,LazyObj(['f','@','v'])))),
                 ['x','w'])})
)
)
) # False

False


In [91]:
print(
show(
RecType({'x' : FunType(Ind,RecTy),
         'y' : Ind,
         'w' : Ind,
         'z' : (Fun('f',FunType(Ind,RecTy), Fun('v',Ind,SingletonType(RecTy,LazyObj(['f','@','v'])))),
                 ['x','y'])}).
create_hypobj(
)
)
)

{x = h76, y = h77, w = h78, z = [h76, @, h77]}


In [92]:
print(
show(
RecType({'x' : FunType(Ind,RecTy),
         'y' : Ind,
         'w' : Ind,
         'z' : (Fun('f',FunType(Ind,RecTy), Fun('v',Ind,SingletonType(RecTy,LazyObj(['f','@','v'])))),
                 ['x','y'])
         })
)
)

{x : (Ind->RecTy), y : Ind, w : Ind, z : (lambda f:(Ind->RecTy) . lambda v:Ind . RecTy_[f, @, v], [x, y])}


In [93]:
print(
    show(
RecType({'x' : FunType(Ind,RecTy),
         'y' : Ind,
         'w' : Ind,
         'z' : (Fun('f',FunType(Ind,RecTy), Fun('v',Ind,SingletonType(RecTy,LazyObj(['f','@','v'])))),
                 ['x','y'])
         }).
subtype_of(RecType({'z' : RecTy}))
)
)

True


In [94]:
print(
    show(
RecType({'x' : FunType(Ind,RecTy),
         'y' : Ind,
         'w' : Ind,
         'z' : (Fun('f',FunType(Ind,RecTy), Fun('v',Ind,SingletonType(RecTy,LazyObj(['f','@','v'])))),
                 ['x','y'])
         }).
         create_hypobj()
         )
         )

{x = h82, y = h83, w = h84, z = [h82, @, h83]}


In [95]:
print(
    show(
        RecType({'x' : FunType(Ind,RecTy),
         'y' : Ind,
         'w' : Ind,
         'z' : (Fun('f',FunType(Ind,RecTy), Fun('v',Ind,SingletonType(RecTy,LazyObj(['f','@','v'])))),
                 ['x','y'])
         }).
         subtype_of(RecType({'z' : RecTy}))
         )
         )

True


In [96]:
TT1 = FunType(Ind, RecTy)
TT2 = FunType(Ind, Ty)
TT3 = FunType(TT1, TT2)
ff = Fun('f', TT1, Fun('x', Ind, LazyObj(['f','@','x'])))
print(TT3.query(ff))

True


#### Absolute paths

In [97]:
print(
Ty.query(RecType({'c' : (Fun('v', Ind, PType(man,['v'])),
                       [AbsPath(Rec({'x' : 'j'}), 'x')])}))
                       )

True


In [98]:
TxInd = RecType({'x' : Ind})
print(
    FunType(TxInd, RecTy).
    query(Fun('r', TxInd, RecType({'c' : (Fun('v', Ind, PType(man,['v'])),
                        [AbsPath('r', 'x')])})))
                       )

True


In [99]:
print(
    show(
        Fun('r', TxInd, RecType({'c' : (Fun('v', Ind, PType(man,['v'])),
                            [AbsPath('r', 'x')])})).app(Rec({'x' : 'j'}))
                        ))

{c : (lambda v:Ind . man(v), [{x = j}.x])}


#### Merging record types

In [100]:
print(
    show(
        RecType({'x' : Ind,
                 'y' : Ind})
        .merge(RecType({'x':Ind}))
        )
        )

{x : Ind, y : Ind}


In [101]:
print(
    show(
     RecType({'x':Ind}).
     merge(RecType({'x' : Ind,
                    'y' : Ind}))
                    )
                    )


{x : Ind, y : Ind}


##### Merging not only for record types

In [102]:
print(
    show(
        JoinType(Ind,Ty).merge(RecTy)))


RecTy


In [103]:
print(
    show(
        JoinType(Ind,Ty).merge(Ind)
    )
    )

Ind


In [104]:
print(
    show(
        JoinType(Ind,Ty).merge(JoinType(Ind,RecTy))
        )
        )

(IndvRecTy)


In [105]:
print(
    show(
        Ind.merge(JoinType(Ind,Ty))
        ))


Ind


#### Tricky cases of merging

In [106]:
print(
    show(
        RecType({'a' : Ind}).merge(RecType({'b' : Ind}))
        )
        )

{a : Ind, b : Ind}


In [107]:
print(
    show(
        RecType({'a' : Ind,
                 'c' : JoinType(Ind,Ty)}).merge(RecType({'b' : Ind,
                                                         'c' : Ind}))
                                                         )
                                                         )

{c : Ind, a : Ind, b : Ind}


In [108]:
T1 = RecType({'x' : Ind,
              'y' : (Fun('v', Ind, RecType({'a' : Ind,
                                            'c' : PType(man, ['v'])})), ['x']),
              'z' : Ind})
T2 = RecType({'x' : Ind,
              'y' : (Fun('v', Ind, RecType({'a' : Ind,
                                            'c' : PType(man, ['v'])})), ['x']),
              'w' : Ind})
print(
    show(
        T1.merge(T2)
        )
        )

{x : Ind, y : (lambda v0:Ind . {a : Ind, c : man(v0)}, [x]), z : Ind, w : Ind}


In [109]:
T1 = RecType({'x' : Ind,
              'y' : (Fun('v', Ind, RecType({'a' : Ind,
                                            'c' : PType(man, ['v'])})), ['x']),
              'z' : Ind})
T2 = RecType({'x' : Ind,
              'y' : (Fun('v', Ind, RecType({'a' : Ind,
                                            'c' : PType(man, ['v'])})), ['z']),
              'z' : Ind})
print(
    show(
        T1.merge(T2)
        )
        )


{x : Ind, y : (lambda v1:Ind . lambda v2:Ind . {a : Ind, c : (man(v1)&man(v2))}, [x, z]), z : Ind}


In [110]:
T1 = RecType({'x' : SingletonType(Ind, 'j'),
              'y' : (Fun('v', Ind, RecType({'a' : Ind,
                                            'c' : PType(man, ['v'])})), ['x']),
              'z' : Ind})
T2 = RecType({'x' : Ind,
              'y' : (Fun('v', Ind, RecType({'a' : Ind,
                                            'c' : PType(man, ['v'])})), ['z']),
              'z' : SingletonType(Ind, 'j')})
print(
    show(
        T1.merge(T2)
        )
        )    

{x : Ind_j, y : (lambda v3:Ind . lambda v4:Ind . {a : Ind, c : (man(v3)&man(v4))}, [x, z]), z : Ind_j}


 In the previous Oz implementation there was an operation that would simplify
 dependent types when the path associated with them pointed to a singleton
 type.  Not (yet) implemented here.

In [111]:
T1 = RecType({'a' : Ind,
              'b' : Ind})
T2 = RecType({'a' : SingletonType(Ind, 'j')})
print(
    show(
        T1.merge(T2)
        )
        )

{a : Ind_j, b : Ind}


In [112]:
T1 = Re
T2 = RecType({'a' : Ind})
print(
    show(
        T1.merge(T2)
        )
        )

{a : Ind}


In [113]:
T1 = RecType({'a' : PType(man, ['j'])})
T2 = RecType({'a' : PType(man, ['m'])})
print(
    show(
        T1.merge(T2)
        )
        )

{a : (man(j)&man(m))}


In [114]:
T1 = RecType({'x' : Ind,
              'e' : (Fun('v',Ind,PType(man,['v'])),['x'])})
T2 = RecType({'e' : PType(man,['j'])})
print(
    show(
        T1.merge(T2)
        )
        )
print(
    show(
        T2.merge(T1)
        )
        )

{e : (lambda v5:Ind . (man(v5)&man(j)), [x]), x : Ind}
{e : (lambda v6:Ind . (man(j)&man(v6)), [x]), x : Ind}


#### Asymmetric merge

In [115]:
T1 = Ind
T2 = RecTy
print(
    show(
        T1.amerge(T2)
        )
        )


RecTy


In [116]:
T1 = RecType({'agenda' : SingletonType(ListType(Ty), [RecType({'e' : PType(run,['j'])})]),
              'latest_move' : SingletonType(ListType(Re),[])})
T2 = RecType({'agenda' : SingletonType(ListType(Ty),[]),
              'latest_move' : RecType({'e' : PType(run,['j'])})})
print(
    show(
        T1.amerge(T2)
        )
        )      

{agenda : [Ty]_[], latest_move : {e : run(j)}}


In [117]:
T1 = RecType({'x' : Ind,
              'agenda' : (Fun('v',Ind,SingletonType(ListType(Ty), [RecType({'e' : PType(run,['v'])})])),['x']),
              'latest_move' : SingletonType(ListType(Re),[])})
T2 = RecType({'x' : Ind,
              'agenda' : SingletonType(ListType(Ty),[]),
              'latest_move' : (Fun('v',Ind,RecType({'e' : PType(run,['v'])})),['x'])})
print(
    show(
        T1.amerge(T2)
        )
        )

{x : Ind, agenda : [Ty]_[], latest_move : (lambda v:Ind . {e : run(v)}, [x])}


In [118]:
T1 = RecType({'x' : Ind,
              'e' : (Fun('v',Ind,PType(run,['v'])),['x'])})
T2 = RecType({'x' : Ind,
              'e' : (Fun('v',Ind,PType(man,['v'])),['x'])})
print(
    show(
        T1.merge(T2)
        )
        )
print(
    show(
        T1.amerge(T2)
        )
        )

{x : Ind, e : (lambda v7:Ind . (run(v7)&man(v7)), [x])}
{x : Ind, e : (lambda v:Ind . man(v), [x])}


In [119]:
T1 = RecType({'x' : Ind,
              'e' : (Fun('v',Ind,PType(run,['v'])),['x'])})
T2 = RecType({'y' : Ind,
              'e' : (Fun('v',Ind,PType(man,['v'])),['y'])})
print(
    show(
        T1.merge(T2)
        )
        )
print(
    show(
        T1.amerge(T2)
        )
        )

{e : (lambda v8:Ind . lambda v9:Ind . (run(v8)&man(v9)), [x, y]), x : Ind, y : Ind}
{e : (lambda v:Ind . man(v), [y]), x : Ind, y : Ind}


In [120]:
T1 = RecType({'x' : Ind,
              'e' : (Fun('v',Ind,RecType({'y' : Ind,
                                          'e' : PType(run,['v'])})),['x'])})
T2 = RecType({'y' : Ind,
              'e' : (Fun('v',Ind,PType(man,['v'])),['y'])})
print(
    show(
        T1.merge(T2)
        )
        )
print(
    show(
        T1.amerge(T2)
        )
        )


{e : (lambda v10:Ind . lambda v11:Ind . ({y : Ind, e : run(v10)}&man(v11)), [x, y]), x : Ind, y : Ind}
{e : (lambda v:Ind . man(v), [y]), x : Ind, y : Ind}


In [121]:
T1 = RecType({'x' : Ind,
              'e' : (Fun('v',Ind,RecType({'y' : Ind,
                                          'e' : PType(run,['v'])})),['x'])})
T2 = RecType({'y' : Ind,
              'e' : (Fun('v',Ind,RecType({'x' : Ind,
                                          'e' : PType(man,['v'])})),['y'])})
print(
    show(
        T1.merge(T2)
        )
        )
print(
    show(
        T1.amerge(T2)
        )
        )

{e : (lambda v12:Ind . lambda v13:Ind . {e : (run(v12)&man(v13)), y : Ind, x : Ind}, [x, y]), x : Ind, y : Ind}
{e : (lambda v:Ind . {x : Ind, e : man(v)}, [y]), x : Ind, y : Ind}


# String types

In [122]:
s = TTRString(['e1','e2'])
print(show(s))

"e1 e2"


In [123]:
T = TTRStringType([PType(run,['j']),PType(run,['m'])])
print(show(T))
print(T.validate())
ttrace()
print(show(T.learn_witness_condition(lambda x: x)))
nottrace()
print(show(T.create()))
print(show(T.create_hypobj()))


run(j)^run(m)
True
run(j)^run(m) is a logical type and cannot learn new conditions
None
"_e8 _e9"
"h128 h129"


In [124]:
KPlusStringType(TTRStringType([RecTy])).query(TTRString([RecType({'e' : PType(run,['j'])}),
                           RecType({'e' : PType(run,['m'])})]))


True

In [125]:
T = KPlusStringType(TTRStringType([RecTy]))
print(show(T))
print(
    show(
        T.query(TTRString([RecType({'e' : PType(run,['j'])}),
                           RecType({'e' : PType(run,['m'])})]))
        )
    )
print(T.validate())
ttrace()
print(show(T.learn_witness_condition(lambda x: x)))
nottrace()
print(show(T.create()))
print(show(T.create_hypobj()))


RecTy+
True
True
RecTy+ is a logical type and cannot learn new conditions
None
_sigma0
h130


In [126]:
T1 = TTRStringType([Re,Re])
T2 = KPlusStringType(TTRStringType([Re]))
print(show(T1.subtype_of(T2)))
print(show(T2.subtype_of(T1)))

True
False


In [127]:
T1 = TTRStringType([Re,Re])
T2 = TTRStringType([RecType({'x' : Ind}), RecType({'y' : Ind})])
print(show(T1.subtype_of(T2)))
print(show(T2.subtype_of(T1)))

False
True


In [128]:
T1 = KPlusStringType(TTRStringType([Re]))
T2 = TTRStringType([RecType({'x' : Ind}), RecType({'y' : Ind})])
print(show(T1.subtype_of(T2)))
print(show(T2.subtype_of(T1)))

False
True


In [129]:
f = Fun('v',Ty,TTRStringType([RecType({'x' : 'v'}), RecType({'y' : 'v'})]))
print(show(f.app(Ind)))

{x : Ind}^{y : Ind}


In [130]:
f = Fun('v',Ty,KPlusStringType(TTRStringType([RecType({'x' : 'v'})])))
print(show(f.app(Ind)))


{x : Ind}+


In [131]:
T1 = TTRStringType([RecType({'x' : Ind}), Re])
T2 = TTRStringType([Re, RecType({'y' : Ind})])
print(show(T1.merge(T2)))
print(show(T1.amerge(T2)))

{x : Ind}^{y : Ind}
{x : Ind}^{y : Ind}


In [132]:
T1 = TTRStringType([RecType({'x' : Ind}), RecType({'x' : Ind})])
T2 = TTRStringType([Re, RecType({'x' : Ty})])
print(show(T1.merge(T2)))
print(show(T1.amerge(T2)))

{x : Ind}^{x : (Ind&Ty)}
{x : Ind}^{x : Ty}


In [133]:
T1 = KPlusStringType(TTRStringType([RecType({'x':Ind})]))
T2 = KPlusStringType(TTRStringType([Re]))
print(show(T1.merge(T2)))
print(show(T1.amerge(T2)))

{x : Ind}+
{x : Ind}+


In [134]:
T1 = KPlusStringType(TTRStringType([RecTy]))
T2 = KPlusStringType(TTRStringType([Ty]))
print(show(T1.merge(T2)))
print(show(T1.amerge(T2)))

RecTy+
RecTy+


In [135]:
T1 = TTRStringType([RecType({'x' : Ind}), RecType({'x' : Ind})])
T2 = KPlusStringType(TTRStringType([RecType({'x':Ind})]))
print(show(T1.merge(T2)))
print(show(T1.amerge(T2)))

{x : Ind}^{x : Ind}
{x : Ind}^{x : Ind}


In [136]:
T1 = TTRStringType([RecType({'x' : Ind}), RecType({'x' : Ind})])
T2 = KPlusStringType(TTRStringType([Ty]))
print(show(T1.merge(T2)))
print(show(T1.amerge(T2)))


({x : Ind}^{x : Ind}&Ty+)
Ty+
